In [ ]:
import sys
sys.path.append("..")

from pandas import DataFrame
import pandas as pd
import numpy as np
import scipy.stats as st
import scipy as sp
from jmetal.lab.statistical_test.critical_distance import CDplot
import matplotlib.pyplot as plt
import random
import re
import os
import git
from scipy.stats import wilcoxon
from sqlalchemy import create_engine, or_, Column, Integer, String, Float, DateTime, ForeignKey, Boolean
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship, sessionmaker
from do_analysis import Experiment, ExperimentSet

In [ ]:
def from_db_to_pandas(query):
    last = ""
    headers = []
    datasets = []
    series = []
    serie = None
    code_directory = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
    repo = git.Repo(code_directory)

    for experiment in query:
        if last != experiment.file_name:
            last = experiment.file_name
            dataset_name = experiment.file_name.rsplit('/')[-1].split(".")[0]
            datasets.append(dataset_name)
            if serie is not None:
                series.append(serie)
            serie = []
        message = experiment.set.description
        measure =  experiment.method.split('.')[-1]
       
        if "Initial" not in experiment.set.description:
            header = measure.replace('Modified','')
        else:
            header = measure
        
        if experiment.method == "weka.core.LearningBasedDissimilarity -R first-last -S E -W K":
            continue
            
        if header not in headers:
            headers.append(header)
        if experiment.number_of_classes is None or experiment.number_of_clusters != experiment.number_of_classes:
            serie.append(0.0)
        else:
            serie.append(experiment.f_score)
    
    series.append(serie)
    print(len(headers))
    
    return pd.DataFrame(series, index = datasets, columns=headers)

In [ ]:
engine = create_engine('sqlite:///../results_testing.db', echo=False)
session_class = sessionmaker(bind=engine)
session = session_class()
query = session.query(Experiment).order_by(Experiment.file_name, Experiment.id)
#query = for experiment in session.query(Experiment).filter(or_(Experiment.set_id==i for i in [17])).order_by(Experiment.file_name):
#query = for experiment in session.query(Experiment).filter(Experiment.number_of_clusters == Experiment.number_of_classes).order_by(Experiment.file_name):
df = from_db_to_pandas(query)
df

In [ ]:
dft = df.apply(lambda x: x.rank(ascending=False), axis = 1)
dft
averages = dft.mean() 
averages = averages.sort_values()
averages.sort_values()

In [ ]:
CDplot(df.transpose(), higher_is_better=True, alpha=0.1, output_filename='cdplotss.png')

In [ ]:
lb = "Lin_KappaMax"
pairs = [x for x in df.columns if x != lb ]
for y in pairs:
    c, p = wilcoxon( df[lb], df[y], alternative="greater")
    if p < 0.1:    
        print(f"{lb} vs {y} -> {p}")
    else:
        print(f"{lb} and {y} are not different, p -> {p}")

In [ ]:
def finner_test_mine(ranks, control=None):
    k = len(ranks)
    values = list(ranks.values())
    keys = list(ranks.keys())
    if not control :
        control_i = values.index(min(values))
    else:
        control_i = keys.index(control)

    comparisons = [keys[control_i] + " vs " + keys[i] for i in range(k) if i != control_i]
    z_values = [abs(values[control_i] - values[i]) for i in range(k) if i != control_i]
    p_values = [2*(1-st.norm.cdf(abs(z))) for z in z_values]
    # Sort values by p_value so that p_0 < p_1
    p_values, z_values, comparisons = map(list, zip(*sorted(zip(p_values, z_values, comparisons), key=lambda t: t[0])))
    adj_p_values = [min(max(1-(1-p_values[j])**((k-1)/float(j+1)) for j in range(i+1)), 1) for i in range(k-1)]
    
    return comparisons, z_values, p_values, adj_p_values


def friedman_test_mine(rankings):
    n = rankings.shape[0]
    k = rankings.shape[1]
    rankings_avg = rankings.mean()
    rankings_cmp = [r/sp.sqrt(k*(k+1)/(6.*n)) for r in rankings_avg]

    chi2 = ((12*n)/float((k*(k+1))))*((sp.sum(r**2 for r in rankings_avg))-((k*(k+1)**2)/float(4)))
    iman_davenport = ((n-1)*chi2)/float((n*(k-1)-chi2))

    p_value = 1 - st.f.cdf(iman_davenport, k-1, (k-1)*(n-1))

    return iman_davenport, p_value, rankings_avg, rankings_cmp

In [ ]:
friedman_res = friedman_test_mine(dft)
f_value, p_value, rankings, pivots = friedman_res
names = df.columns
avg_dic = {name: pivot for name, pivot in zip(names, pivots)}
t = finner_test_mine(avg_dic)
comparisons, z_values, p_values, adjusted_p_values = t
#t

In [ ]:
dft_dic = {}
for i in dft.columns:
    values = [x for x in dft[i]]
    dft_dic[i] = values

avg_dic = averages.to_dict()
avg_dic
#dft_dic.keys()
t = finner_test_mine(avg_dic)
comparisons, z_values, p_values, adjusted_p_values = t
t